In [1]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
import random
from glob import glob, iglob
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np; print('numpy', np.__version__)
import pandas as pd; print('pandas', pd.__version__)
import cv2; print('opencv2', cv2.__version__)
import sklearn; print('sklearn', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import tensorflow.keras as keras; print('keras', keras.__version__)

import settings
import helper

Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]] on linux
numpy 1.16.4
pandas 0.24.2
opencv2 4.1.0
sklearn 0.21.2
tensorflow 1.14.0
keras 2.2.4-tf


# 1. Load Meta File and Annotation

In [2]:
df_meta_train = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TRAIN_META_FILE, index_col=['seriesuid'])
df_meta_train.index = df_meta_train.index.astype('str')
df_meta_test = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TEST_META_FILE, index_col=['seriesuid'])
df_meta_test.index = df_meta_test.index.astype('str')

print('meta_train:', df_meta_train.shape, '\n', 'meta_test:', df_meta_test.shape)

meta_train: (1470, 11) 
 meta_test: (222, 11)


In [3]:
df_annotation = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_ANNOTATION_FILE, index_col=['seriesuid'])
df_annotation.index = df_annotation.index.astype('str')
df_annotation['label'] = df_annotation['label'].astype('int')
print('annotation:', df_annotation.shape, 'distinct lung:', len(set(df_annotation.index)))

annotation: (12218, 16) distinct lung: 1436


In [4]:
df_annotation.sample(10)

,width,height,slice,vcoordX,vcoordY,vcoordZ,diameterX,diameterY,diameterZ,originX,originY,originZ,spacingX,spacingY,spacingZ,label
seriesuid,,,,,,,,,,,,,,,,
697842,350,350,309,104.248047,74.511719,129.0,6.835938,7.519531,3.0,-176.050003,-14.800000,-265.350006,0.683594,0.683594,3.0,1
688571,350,350,235,254.980469,189.355469,135.0,7.519531,11.621094,5.0,-174.658203,-174.658203,-500.765747,0.683594,0.683594,5.0,1
631024,274,274,330,211.654297,75.189453,88.0,13.914062,8.562500,10.0,-129.732422,-276.732422,-252.399994,0.535156,0.535156,5.0,5
690557,350,350,310,111.083984,189.013672,270.0,6.835938,10.937500,5.0,-176.110809,-174.658203,332.549194,0.683594,0.683594,5.0,1
402074,370,370,210,109.121058,248.593668,133.0,5.058592,7.949216,7.0,-189.199997,-185.000000,-821.789978,0.722656,0.722656,7.0,1
645480,350,350,295,58.105469,177.392578,143.0,7.519531,8.203125,10.0,-170.300415,-174.658203,-190.553223,0.683594,0.683594,5.0,1
688322,350,350,380,100.488281,155.175781,90.0,11.621094,11.621094,5.0,-181.921158,-174.658203,-556.596436,0.683594,0.683594,5.0,5
388757,309,309,280,147.257812,110.745117,195.0,12.673828,14.484375,5.0,-141.198242,-300.198242,-883.500000,0.603516,0.603516,5.0,31
655923,361,361,260,257.000977,125.503906,135.0,5.640625,9.166016,5.0,-167.800446,-180.147461,-322.552246,0.705078,0.705078,5.0,1


# 2. Load Center Coords by Candidates

In [5]:
lungs = list(set(df_meta_train.index))
random.shuffle(lungs)
print('distinct lungs:', len(lungs))

distinct lungs: 1470


In [6]:
list_tasks = []
size = settings.CUBE_SIZE
step = 32
start = [ size//2, size//2, size//2 ] # z,y,x
limit = 100

count = 0
for uid in tqdm(lungs):
    meta = df_meta_train.loc[uid] # slice
    
    for z in range(start[0], int(meta.slice), step):
        for y in range(start[1], int(meta.height), step):
            for x in range(start[2], int(meta.width), step):
                task = {}
                task['seriesuid'] = uid
                task['vcoordX'] = x
                task['vcoordY'] = y
                task['vcoordZ'] = z
                
                # print(task)
                list_tasks.append(task)
    
    count += 1
    
    if count >= limit:
        break

  5%|▍         | 69/1470 [00:00<00:04, 345.95it/s]


In [7]:
# df_tasks = pd.DataFrame(list_tasks, columns=['seriesuid','vcoordX','vcoordY','vcoordZ'])
# df_tasks = df_tasks.set_index('seriesuid')
# df_tasks.index = df_tasks.index.astype('str')

# print('total:', df_tasks.shape, 'lung:', len(set(df_tasks.index)))

# if not os.path.exists(settings.SUBMISSION_DIR + 'train'):
#     os.mkdir(settings.SUBMISSION_DIR + 'train')
    
# df_tasks.to_csv(settings.SUBMISSION_DIR + 'train' + '/tasks_cubes_medi.csv', encoding='utf-8')


In [8]:
df_tasks = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'train' + '/tasks_cubes_medi.csv', index_col=['seriesuid'])
df_tasks.index = df_tasks.index.astype('str')

print('total:', df_tasks.shape, 'distinct lung:', len(set(df_tasks.index)))

total: (94702, 3) distinct lung: 100


# 3. Predict Results by tasks_cubes.csv

In [9]:
# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()

# gpu allow_growth
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.Session(config=config))

In [10]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    SMOOTH = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + SMOOTH) / (K.sum(y_true_f) + K.sum(y_pred_f) + SMOOTH)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

label = '31'
output_dir = './output/unet3d/1564409330/' # labels in center coord
model = load_model(output_dir + "model-best.hd5", 
                   custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})

W0730 14:37:06.151365 140431665747776 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0730 14:37:06.152518 140431665747776 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0730 14:37:06.158895 140431665747776 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprec

In [11]:
batch_size = 64
threshold_probability = 0.8
wtype = 'medi'
plot = False

In [12]:
def plot_cube(cube, rows=8, cols=8):
    num_z, h, w = cube.shape # z,y,x
    assert rows * cols == num_z
    
    img = np.zeros((rows * h, cols * w), dtype=np.uint8)

    for row in range(rows):
        for col in range(cols):
            y = row * h
            x = col * w
            img[y:(y+h), x:(x+w)] = cube[row * cols + col,:,:]
    
    fig, axs = plt.subplots(1, 1, figsize=(32, 32), sharex=True, sharey=True)
    axs.imshow(img, cmap='gray')
    plt.show()
    

In [13]:
list_results = []
print(wtype, 'window')
for uid in set(df_tasks.index):
    tasks = df_tasks.loc[[uid]] # DataFrame
    meta = df_meta_train.loc[uid] # slice
    
    print('lung:', uid, 'tasks:', len(tasks), 'meta:', meta.slice, meta.height, meta.width)
#     print(meta)
                 
    lung_l, mask_l = helper.load_lung_array(uid, int(meta.width), int(meta.height), int(meta.slice), 
                                            mode='train', wtype=wtype)

    lung = (lung_l*(mask_l>0)).astype('float') / 255

    count = 0
    for idx, item in tasks.iterrows():
        if count % (batch_size*100) == 0:
            X_item = []
            X_test = []

        # patch without mask
        cube = helper.get_cube_from_lung_array(lung, item.vcoordX, item.vcoordY, item.vcoordZ, 
                                               block_size=settings.CUBE_SIZE)

        if np.sum(cube) > 0:
            X_item.append(item)
            X_test.append(cube)
#         else:
#             print('nothing in coord:', '\n', item)

        if (count % (batch_size*100) == (batch_size*100 - 1) or count == len(tasks) - 1) and len(X_test) > 0:
            assert len(X_item) == len(X_test) or count == len(tasks) - 1

            # Make predictions
            X_test = np.expand_dims(np.asarray(X_test), axis=-1)
            print(X_test.shape)
            predictions_test = model.predict(X_test, batch_size=batch_size, verbose=1)

            for i in range(len(X_item)):
                itm = X_item[i]
                pred = predictions_test[i] > threshold_probability
                pred_cube = predictions_test[i,:,:,:,0]
                pred_mask = pred[:,:,:,0]
            
                if np.sum(pred) > 10:
                    vcoords, diameters, bboxes = helper.get_regions_detail(pred[:,:,:,0], itm)
                    if len(bboxes) > 0:
                        for i, b in enumerate(bboxes):
                            result = {}
                            result['seriesuid'] = uid
                            result['vcoordX'] = vcoords[i][2]
                            result['vcoordY'] = vcoords[i][1]
                            result['vcoordZ'] = vcoords[i][0]
                            result['diametersX'] = diameters[i][2]
                            result['diametersY'] = diameters[i][1]
                            result['diametersZ'] = diameters[i][0]
                            
                            region = pred_cube[b[0]:b[3],b[1]:b[4],b[2]:b[5]]
                            result['probability'] = region[region>threshold_probability].mean()
                            if plot:
                                plot_cube((pred*255).astype(np.uint8))

                            list_results.append(result)
                        
        count += 1
   
    # save the csv by per lung
    df_results = pd.DataFrame(list_results, columns=['seriesuid','vcoordX','vcoordY','vcoordZ','diametersX','diametersY','diametersZ','probability'])
    df_results = df_results.set_index('seriesuid')
    df_results.index = df_results.index.astype('str')

    print('total:', df_results.shape, 'lung:', len(set(df_results.index)))

    df_results.to_csv(settings.SUBMISSION_DIR + 'train' + '/tasks_' + wtype + '_' + label + '.csv', encoding='utf-8')

    print('-'*100)
    

medi window
lung: 655457 tasks: 567 meta: 230.0 306.0 306.0
(567, 64, 64, 64, 1)
567/567 [==============================] - 18s 32ms/sample
total: (83, 7) lung: 1
----------------------------------------------------------------------------------------------------
lung: 672912 tasks: 1296 meta: 300.0 401.0 401.0
(1199, 64, 64, 64, 1)
1199/1199 [==============================] - 24s 20ms/sample
total: (243, 7) lung: 2
----------------------------------------------------------------------------------------------------
lung: 400058 tasks: 729 meta: 320.0 291.0 291.0
(692, 64, 64, 64, 1)
692/692 [==============================] - 15s 22ms/sample
total: (433, 7) lung: 3
----------------------------------------------------------------------------------------------------
lung: 671752 tasks: 729 meta: 320.0 306.0 306.0
(719, 64, 64, 64, 1)
719/719 [==============================] - 14s 20ms/sample
total: (549, 7) lung: 4
--------------------------------------------------------------------------

(1404, 64, 64, 64, 1)
1404/1404 [==============================] - 28s 20ms/sample
total: (4388, 7) lung: 33
----------------------------------------------------------------------------------------------------
lung: 656403 tasks: 1440 meta: 324.0 394.0 394.0
(1188, 64, 64, 64, 1)
1188/1188 [==============================] - 25s 21ms/sample
total: (4553, 7) lung: 34
----------------------------------------------------------------------------------------------------
lung: 655574 tasks: 968 meta: 280.0 381.0 381.0
(906, 64, 64, 64, 1)
906/906 [==============================] - 18s 20ms/sample
total: (4699, 7) lung: 35
----------------------------------------------------------------------------------------------------
lung: 520069 tasks: 1690 meta: 340.0 427.0 427.0
(1433, 64, 64, 64, 1)
1433/1433 [==============================] - 29s 20ms/sample
total: (4839, 7) lung: 36
----------------------------------------------------------------------------------------------------
lung: 691784 task

(827, 64, 64, 64, 1)
827/827 [==============================] - 18s 22ms/sample
total: (8818, 7) lung: 65
----------------------------------------------------------------------------------------------------
lung: 645337 tasks: 900 meta: 320.0 350.0 350.0
(812, 64, 64, 64, 1)
812/812 [==============================] - 16s 19ms/sample
total: (8960, 7) lung: 66
----------------------------------------------------------------------------------------------------
lung: 502378 tasks: 512 meta: 260.0 280.0 280.0
(481, 64, 64, 64, 1)
481/481 [==============================] - 9s 20ms/sample
total: (9037, 7) lung: 67
----------------------------------------------------------------------------------------------------
lung: 402074 tasks: 726 meta: 210.0 370.0 370.0
(625, 64, 64, 64, 1)
625/625 [==============================] - 14s 23ms/sample
total: (9146, 7) lung: 68
----------------------------------------------------------------------------------------------------
lung: 696418 tasks: 800 meta:

(967, 64, 64, 64, 1)
967/967 [==============================] - 19s 20ms/sample
total: (12964, 7) lung: 97
----------------------------------------------------------------------------------------------------
lung: 639006 tasks: 576 meta: 300.0 280.0 280.0
(571, 64, 64, 64, 1)
571/571 [==============================] - 11s 19ms/sample
total: (13042, 7) lung: 98
----------------------------------------------------------------------------------------------------
lung: 630889 tasks: 800 meta: 270.0 346.0 346.0
(792, 64, 64, 64, 1)
792/792 [==============================] - 15s 20ms/sample
total: (13157, 7) lung: 99
----------------------------------------------------------------------------------------------------
lung: 663638 tasks: 900 meta: 315.0 350.0 350.0
(876, 64, 64, 64, 1)
876/876 [==============================] - 17s 19ms/sample
total: (13277, 7) lung: 100
----------------------------------------------------------------------------------------------------


In [14]:
df_results = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'train' + '/tasks_' + wtype +'.csv', 
                            index_col=['seriesuid'])
df_results.index = df_results.index.astype('str')

print('results:', df_results.shape, 'distinct lung:', len(set(df_results.index)))



results: (165965, 7) distinct lung: 1000


In [15]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    SMOOTH = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + SMOOTH) / (K.sum(y_true_f) + K.sum(y_pred_f) + SMOOTH)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

label = '32'
output_dir = './output/unet3d/1564409333/' # labels in center coord
model = load_model(output_dir + "model-best.hd5", 
                   custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})

In [16]:
list_results = []
print(wtype, 'window')
for uid in set(df_tasks.index):
    tasks = df_tasks.loc[[uid]] # DataFrame
    meta = df_meta_train.loc[uid] # slice
    
    print('lung:', uid, 'tasks:', len(tasks), 'meta:', meta.slice, meta.height, meta.width)
#     print(meta)
                 
    lung_l, mask_l = helper.load_lung_array(uid, int(meta.width), int(meta.height), int(meta.slice), 
                                            mode='train', wtype=wtype)

    lung = (lung_l*(mask_l>0)).astype('float') / 255

    count = 0
    for idx, item in tasks.iterrows():
        if count % (batch_size*100) == 0:
            X_item = []
            X_test = []

        # patch without mask
        cube = helper.get_cube_from_lung_array(lung, item.vcoordX, item.vcoordY, item.vcoordZ, 
                                               block_size=settings.CUBE_SIZE)

        if np.sum(cube) > 0:
            X_item.append(item)
            X_test.append(cube)
#         else:
#             print('nothing in coord:', '\n', item)

        if (count % (batch_size*100) == (batch_size*100 - 1) or count == len(tasks) - 1) and len(X_test) > 0:
            assert len(X_item) == len(X_test) or count == len(tasks) - 1

            # Make predictions
            X_test = np.expand_dims(np.asarray(X_test), axis=-1)
            print(X_test.shape)
            predictions_test = model.predict(X_test, batch_size=batch_size, verbose=1)

            for i in range(len(X_item)):
                itm = X_item[i]
                pred = predictions_test[i] > threshold_probability
                pred_cube = predictions_test[i,:,:,:,0]
                pred_mask = pred[:,:,:,0]
         
                if np.sum(pred) > 10:
                    vcoords, diameters, bboxes = helper.get_regions_detail(pred_mask, itm)
                    if len(bboxes) > 0:
                        # np.save(settings.SUBMISSION_DIR + 'train' + f'/predicted_cube_{i}.npy', predictions_test[i])
                        for i, b in enumerate(bboxes):
                            result = {}
                            result['seriesuid'] = uid
                            result['vcoordX'] = vcoords[i][2]
                            result['vcoordY'] = vcoords[i][1]
                            result['vcoordZ'] = vcoords[i][0]
                            result['diametersX'] = diameters[i][2]
                            result['diametersY'] = diameters[i][1]
                            result['diametersZ'] = diameters[i][0]
                            
                            region = pred_cube[b[0]:b[3],b[1]:b[4],b[2]:b[5]]
                            result['probability'] = region[region>threshold_probability].mean()
                            if plot:
                                plot_cube((pred_mask*255).astype(np.uint8))

                            list_results.append(result)
                        
        count += 1
   
    # save the csv by per lung
    df_results = pd.DataFrame(list_results, columns=['seriesuid','vcoordX','vcoordY','vcoordZ','diametersX','diametersY','diametersZ','probability'])
    df_results = df_results.set_index('seriesuid')
    df_results.index = df_results.index.astype('str')

    print('total:', df_results.shape, 'lung:', len(set(df_results.index)))

    df_results.to_csv(settings.SUBMISSION_DIR + 'train' + '/tasks_' + wtype + '_' + label + '.csv', encoding='utf-8')

    print('-'*100)
    

medi window
lung: 655457 tasks: 567 meta: 230.0 306.0 306.0
(567, 64, 64, 64, 1)
567/567 [==============================] - 13s 23ms/sample
total: (99, 7) lung: 1
----------------------------------------------------------------------------------------------------
lung: 672912 tasks: 1296 meta: 300.0 401.0 401.0
(1199, 64, 64, 64, 1)
1199/1199 [==============================] - 24s 20ms/sample
total: (233, 7) lung: 2
----------------------------------------------------------------------------------------------------
lung: 400058 tasks: 729 meta: 320.0 291.0 291.0
(692, 64, 64, 64, 1)
692/692 [==============================] - 13s 19ms/sample
total: (393, 7) lung: 3
----------------------------------------------------------------------------------------------------
lung: 671752 tasks: 729 meta: 320.0 306.0 306.0
(719, 64, 64, 64, 1)
719/719 [==============================] - 14s 20ms/sample
total: (507, 7) lung: 4
--------------------------------------------------------------------------

(1404, 64, 64, 64, 1)
1404/1404 [==============================] - 27s 20ms/sample
total: (3714, 7) lung: 33
----------------------------------------------------------------------------------------------------
lung: 656403 tasks: 1440 meta: 324.0 394.0 394.0
(1188, 64, 64, 64, 1)
1188/1188 [==============================] - 23s 20ms/sample
total: (3851, 7) lung: 34
----------------------------------------------------------------------------------------------------
lung: 655574 tasks: 968 meta: 280.0 381.0 381.0
(906, 64, 64, 64, 1)
906/906 [==============================] - 17s 19ms/sample
total: (3990, 7) lung: 35
----------------------------------------------------------------------------------------------------
lung: 520069 tasks: 1690 meta: 340.0 427.0 427.0
(1433, 64, 64, 64, 1)
1433/1433 [==============================] - 28s 20ms/sample
total: (4116, 7) lung: 36
----------------------------------------------------------------------------------------------------
lung: 691784 task

(827, 64, 64, 64, 1)
827/827 [==============================] - 16s 20ms/sample
total: (7881, 7) lung: 65
----------------------------------------------------------------------------------------------------
lung: 645337 tasks: 900 meta: 320.0 350.0 350.0
(812, 64, 64, 64, 1)
812/812 [==============================] - 16s 20ms/sample
total: (8004, 7) lung: 66
----------------------------------------------------------------------------------------------------
lung: 502378 tasks: 512 meta: 260.0 280.0 280.0
(481, 64, 64, 64, 1)
481/481 [==============================] - 10s 20ms/sample
total: (8096, 7) lung: 67
----------------------------------------------------------------------------------------------------
lung: 402074 tasks: 726 meta: 210.0 370.0 370.0
(625, 64, 64, 64, 1)
625/625 [==============================] - 13s 20ms/sample
total: (8191, 7) lung: 68
----------------------------------------------------------------------------------------------------
lung: 696418 tasks: 800 meta

967/967 [==============================] - 19s 20ms/sample
total: (11906, 7) lung: 97
----------------------------------------------------------------------------------------------------
lung: 639006 tasks: 576 meta: 300.0 280.0 280.0
(571, 64, 64, 64, 1)
571/571 [==============================] - 11s 20ms/sample
total: (12017, 7) lung: 98
----------------------------------------------------------------------------------------------------
lung: 630889 tasks: 800 meta: 270.0 346.0 346.0
(792, 64, 64, 64, 1)
792/792 [==============================] - 15s 20ms/sample
total: (12122, 7) lung: 99
----------------------------------------------------------------------------------------------------
lung: 663638 tasks: 900 meta: 315.0 350.0 350.0
(876, 64, 64, 64, 1)
876/876 [==============================] - 17s 20ms/sample
total: (12281, 7) lung: 100
----------------------------------------------------------------------------------------------------


In [17]:
df_results = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'train' + '/tasks_' + wtype + '_' + label + '.csv', 
                            index_col=['seriesuid'])
df_results.index = df_results.index.astype('str')

print('results:', df_results.shape, 'distinct lung:', len(set(df_results.index)))


results: (12281, 7) distinct lung: 100


In [18]:
filepath_31 = settings.SUBMISSION_DIR + 'train' + '/tasks_' + wtype +'_31.csv'
filepath_32 = settings.SUBMISSION_DIR + 'train' + '/tasks_' + wtype +'_32.csv'
if os.path.exists(filepath_31) and os.path.exists(filepath_32):
    df_medi_31 = pd.read_csv(filepath_or_buffer=filepath_31, index_col=['seriesuid'])
    df_medi_31.index = df_medi_31.index.astype('str')
    print('medi window 31:', df_medi_31.shape)
    
    df_medi_32 = pd.read_csv(filepath_or_buffer=filepath_32, index_col=['seriesuid'])
    df_medi_32.index = df_medi_32.index.astype('str')
    print('medi window 32:', df_medi_32.shape)
    
    df_medi = pd.concat([df_medi_31, df_medi_32])
    print('medi windows merged:', df_medi.shape)
    df_medi.to_csv(settings.SUBMISSION_DIR + 'train' + '/tasks_' + wtype +'.csv', encoding='utf-8')

medi window 31: (13277, 7)
medi window 32: (12281, 7)
medi windows merged: (25558, 7)
